In [ ]:
import sys
!conda install -c conda-forge --yes --prefix {sys.prefix} hvplot, dask, contextily

In [1]:
import ee
import geemap
import glob
import xarray as xr
import rioxarray as rio

In [2]:
Map = geemap.Map(center=(49.939, 8.5), zoom=16)
Map.add_basemap('HYBRID')
Map

Map(center=[49.939, 8.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

#### Define Geometry

In [3]:
# Draw any shapes on the map using the Drawing tools before executing this code block
feature = Map.draw_last_feature
roi = feature.geometry()

AttributeError: 'NoneType' object has no attribute 'geometry'

In [4]:
roi = ee.Geometry({'geodesic': False,
 'type': 'Polygon',
 'coordinates': [[[8.500447, 49.93948],
   [8.501573, 49.939256],
   [8.50116, 49.938403],
   [8.500044, 49.938624],
   [8.500447, 49.93948]]]})
Map.addLayer(roi)
roi = roi.buffer(10000)
Map.addLayer(roi)

#### Filter Sentinel-1 GRD Collection

In [5]:
s1_collection = ee.ImageCollection("COPERNICUS/S1_GRD") \
    .filterBounds(roi) \
    .filterDate('2016-03-01', '2021-10-31') \
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV")) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .select(['VV', 'angle']) 

#### Add NDVI Band to S1-GRD Collection

In [6]:
def add_ndvi(image, dateoffset = 15):
        """
        Arguments: Filter S2 TOA Collection to roi, mask cloudy pixels, calculate NDVI values, Make mosaic from +- 15 days from s1 image
        """
        def maskS2clouds(image):
            qa = image.select('QA60')
            #Bits 10 and 11 are clouds and cirrus, respectively.
            cloudBitMask = 1 << 10
            cirrusBitMask = 1 << 11
            #Both flags should be set to zero, indicating clear conditions.
            mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
            return image.updateMask(mask).divide(10000)

        def NDVI(image):
            ndvi = image.normalizedDifference(['nir','red']).rename('NDVI') #(first − second) / (first + second)
            return image.addBands(ndvi)
        
        # Sentinel 2 image collection with corresponding named bands
        bandNamesOut_s2 = ['Aerosols','blue','green','red','red edge 1','red edge 2','red edge 3','nir','red edge 4','water vapor','cirrus','swir1','swir2','QA60']
        bandNamesS2 = ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12','QA60']
        
        s2_1c = ee.ImageCollection('COPERNICUS/S2').select(bandNamesS2,bandNamesOut_s2)
        s2_1c = s2_1c.filterDate(ee.Date(image.date().advance(-dateoffset,'days')), ee.Date(image.date().advance(+dateoffset,'days'))).filterBounds(image.geometry()).map(maskS2clouds).map(NDVI)
        
        ndvi = ee.Image(s2_1c.qualityMosaic('NDVI').select('NDVI'))

        return image.addBands(ndvi)

In [7]:
s1_collection = s1_collection.map(add_ndvi)

In [8]:
#Mask out area with invalid landcover class like build up areas
lc = ee.Image(ee.ImageCollection("ESA/WorldCover/v100").first().clip(roi).select('Map'))
s1_collection = s1_collection.map(lambda x: x.addBands(lc))

#### Split Collection into different observations modes to reduce errors

In [9]:
desc = s1_collection.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
senA_desc = desc.filter(ee.Filter.eq('platform_number', 'A'))
senB_desc = desc.filter(ee.Filter.eq('platform_number', 'B'))

asc = s1_collection.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
senA_asc = asc.filter(ee.Filter.eq('platform_number', 'A'))
senB_asc = asc.filter(ee.Filter.eq('platform_number', 'B'))

#### Add Soil Water Index

In [10]:
def add_swi(image):
    swi = image.select('VV').subtract(vvdry_image).divide(vvwet_image.subtract(vvdry_image))
    swi = swi.select('VV').rename('SWI')
    return image.addBands(swi)

In [11]:
vvdry_image = senA_desc.select('VV').reduce(ee.Reducer.percentile([5]))
vvwet_image = senA_desc.select('VV').reduce(ee.Reducer.percentile([95]))
senA_desc = senA_desc.map(add_swi)

vvdry_image = senB_desc.select('VV').reduce(ee.Reducer.percentile([5]))
vvwet_image = senB_desc.select('VV').reduce(ee.Reducer.percentile([95]))
senB_desc = senB_desc.map(add_swi)

vvdry_image = senA_asc.select('VV').reduce(ee.Reducer.percentile([5]))
vvwet_image = senA_asc.select('VV').reduce(ee.Reducer.percentile([95]))
senA_asc = senA_asc.map(add_swi)

vvdry_image = senB_asc.select('VV').reduce(ee.Reducer.percentile([5]))
vvwet_image = senB_asc.select('VV').reduce(ee.Reducer.percentile([95]))
senB_asc = senB_asc.map(add_swi)

#### Visualizing  time series  data

In [ ]:
vis_params_VV = {
  'min': -30,
  'max': 0,
  'palette': ['red', 'blue']
}
vis_params_NDVI = {
  'min': 0,
  'max': 1,
  'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301']
}
vis_params_SWI = {
  'min': 0,
  'max': 1,
  'palette': ['red','orange','lightblue', 'blue']
}

In [ ]:
Map = geemap.Map(center=(49.75, 8.5), zoom=10)
Map.add_basemap('HYBRID')
image = senA_desc.select('NDVI').toBands()
Map.addLayer(image, {}, "Time series NDVI - s2 toa", False)
labels = [str(n).zfill(2) for n in range(senA_desc.size().getInfo())]
Map.add_time_slider(senA_desc.select('NDVI'), vis_params_NDVI, labels=labels, time_interval=1)
Map

## Download an ee.ImageCollection

In [12]:
#clip collection to roi extend and export images
geemap.ee_export_image_collection(senA_desc, out_dir='C://Users/USER/Downloads/gee_downloads/senA_desc', region=roi, file_per_band=True, scale=50)

Total number of images: 338

Exporting 1/338: S1A_IW_GRDH_1SDV_20160302T054202_20160302T054227_010186_00F082_9C77.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 2/338: S1A_IW_GRDH_1SDV_20160309T053351_20160309T053416_010288_00F382_97E9.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 3/338: S1A_IW_GRDH_1SDV_20160314T054155_20160314T054220_010361_00F59F_7AED.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 4/338: S1A_IW_GRDH_1SDV_20160321T053402_20160321T053427_010463_00F877_8A74.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 5/338: S1A_IW_GRDH_1SDV_20160326T054202_20160326T054227_010536_00FA77_AE59.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Export

Exporting 23/338: S1A_IW_GRDH_1SDV_20160731T053344_20160731T053415_012388_013517_B8EC.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 24/338: S1A_IW_GRDH_1SDV_20160805T054153_20160805T054225_012461_013783_1087.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 25/338: S1A_IW_GRDH_1SDV_20160812T053409_20160812T053434_012563_013AED_8A30.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 26/338: S1A_IW_GRDH_1SDV_20160817T054154_20160817T054225_012636_013D40_FF7A.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 27/338: S1A_IW_GRDH_1SDV_20160824T053410_20160824T053435_012738_0140C6_1E42.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 28/338: S1A_IW_GRDH_

Exporting 45/338: S1A_IW_GRDH_1SDV_20161210T053410_20161210T053435_014313_0172CD_59DA.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 46/338: S1A_IW_GRDH_1SDV_20161215T054219_20161215T054244_014386_017511_A9F1.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 47/338: S1A_IW_GRDH_1SDV_20161222T053410_20161222T053435_014488_017843_A013.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 48/338: S1A_IW_GRDH_1SDV_20161227T054219_20161227T054244_014561_017A7E_FC21.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 49/338: S1A_IW_GRDH_1SDV_20170103T053408_20170103T053433_014663_017DA3_2B6D.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 50/338: S1A_IW_GRDH_

Exporting 67/338: S1A_IW_GRDH_1SDV_20170421T053409_20170421T053434_016238_01AD9E_63A1.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 68/338: S1A_IW_GRDH_1SDV_20170426T054218_20170426T054243_016311_01AFCB_2EB1.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 69/338: S1A_IW_GRDH_1SDV_20170503T053409_20170503T053434_016413_01B2EC_86B3.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 70/338: S1A_IW_GRDH_1SDV_20170508T054219_20170508T054244_016486_01B519_9E6C.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 71/338: S1A_IW_GRDH_1SDV_20170515T053410_20170515T053435_016588_01B83E_B568.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 72/338: S1A_IW_GRDH_

Exporting 89/338: S1A_IW_GRDH_1SDV_20170905T054220_20170905T054245_018236_01EA82_F6DC.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 90/338: S1A_IW_GRDH_1SDV_20170912T053357_20170912T053422_018338_01EDBF_9251.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 91/338: S1A_IW_GRDH_1SDV_20170917T054220_20170917T054245_018411_01EFF6_B8CC.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 92/338: S1A_IW_GRDH_1SDV_20170924T053357_20170924T053422_018513_01F31D_2B30.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 93/338: S1A_IW_GRDH_1SDV_20170929T054220_20170929T054245_018586_01F54B_76F0.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 94/338: S1A_IW_GRDH_

...

Exporting 243/338: S1A_IW_GRDH_1SDV_20200405T053408_20200405T053433_031988_03B1D4_046B.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 244/338: S1A_IW_GRDH_1SDV_20200410T054231_20200410T054256_032061_03B471_FA45.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 245/338: S1A_IW_GRDH_1SDV_20200417T053408_20200417T053433_032163_03B803_85EA.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 246/338: S1A_IW_GRDH_1SDV_20200422T054231_20200422T054256_032236_03BA93_EE33.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 247/338: S1A_IW_GRDH_1SDV_20200429T053409_20200429T053434_032338_03BE26_B9BB.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 248/338: S1A_IW

Exporting 265/338: S1A_IW_GRDH_1SDV_20200815T053415_20200815T053440_033913_03EF0C_A6C9.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 266/338: S1A_IW_GRDH_1SDV_20200820T054238_20200820T054303_033986_03F1A3_EEE9.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 267/338: S1A_IW_GRDH_1SDV_20200827T053416_20200827T053441_034088_03F535_D1CB.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 268/338: S1A_IW_GRDH_1SDV_20200901T054239_20200901T054304_034161_03F7D2_0B57.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 269/338: S1A_IW_GRDH_1SDV_20200908T053416_20200908T053441_034263_03FB5F_03BD.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 270/338: S1A_IW

Exporting 287/338: S1A_IW_GRDH_1SDV_20201225T053415_20201225T053440_035838_043216_26BA.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 288/338: S1A_IW_GRDH_1SDV_20201230T054238_20201230T054303_035911_0434B3_96C7.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 289/338: S1A_IW_GRDH_1SDV_20210106T053415_20210106T053440_036013_043833_64B6.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 290/338: S1A_IW_GRDH_1SDV_20210111T054237_20210111T054302_036086_043ACA_212A.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 291/338: S1A_IW_GRDH_1SDV_20210118T053414_20210118T053443_036188_043E55_3E5D.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 292/338: S1A_IW

Exporting 309/338: S1A_IW_GRDH_1SDV_20210506T053415_20210506T053440_037763_0474F3_26B8.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 310/338: S1A_IW_GRDH_1SDV_20210511T054238_20210511T054303_037836_047737_BEAF.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 311/338: S1A_IW_GRDH_1SDV_20210518T053416_20210518T053441_037938_047A3A_1159.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 312/338: S1A_IW_GRDH_1SDV_20210523T054239_20210523T054304_038011_047C7E_651B.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 313/338: S1A_IW_GRDH_1SDV_20210530T053416_20210530T053441_038113_047F7E_1B20.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 314/338: S1A_IW

Exporting 331/338: S1A_IW_GRDH_1SDV_20210915T053422_20210915T053447_039688_04B16E_1CB2.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 332/338: S1A_IW_GRDH_1SDV_20210920T054245_20210920T054310_039761_04B3D0_987B.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 333/338: S1A_IW_GRDH_1SDV_20210927T053423_20210927T053448_039863_04B74D_2A48.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 334/338: S1A_IW_GRDH_1SDV_20211002T054246_20211002T054311_039936_04B9DC_8DEF.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 335/338: S1A_IW_GRDH_1SDV_20211009T053423_20211009T053448_040038_04BD5B_C008.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senA_desc


Exporting 336/338: S1A_IW

In [ ]:
geemap.ee_export_image_collection(senB_desc.select(['VV','Map', 'NDVI']) , out_dir='C://Users/USER/Downloads/gee_downloads/senB_desc', region=roi, file_per_band=True, scale=50)

Total number of images: 279

Exporting 1/279: S1B_IW_GRDH_1SDV_20160928T054135_20160928T054200_002265_003D06_4CB8.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\USER\Downloads\gee_downloads\senB_desc


Exporting 2/279: S1B_IW_GRDH_1SDV_20170310T053310_20170310T053335_004642_008186_63F5.tif
Generating URL ...


In [ ]:
geemap.ee_export_image_collection(senA_asc.select(['VV','Map', 'NDVI']), out_dir='C://Users/USER/Downloads/gee_downloads/senA_asc', region=roi, file_per_band=True, scale=50)

In [ ]:
geemap.ee_export_image_collection(senB_asc.select(['VV','Map', 'NDVI']), out_dir='C://Users/USER/Downloads/gee_downloads/senB_asc', region=roi, file_per_band=True, scale=50)

#### Open files in xarray

In [ ]:
files = glob.glob('C://Users/USER/Downloads/gee_downloads/senA_desc/*.tif')

In [ ]:
from datetime import datetime

In [ ]:
def preprocess_s1(da):
    da = da.drop('spatial_ref').sel(band=1).drop('band')
    filename = da.encoding["source"]
    variable = filename.split('.')[-2]
    date = datetime.strptime(filename.split('_')[6], '%Y%m%dT%H%M%S')
    da['time'] = date
    da = da.set_coords('time').expand_dims('time')
    da = da.isel(x=slice(0,64), y=slice(0,44))
    if variable == 'angle':
        da = da.rename({'band_data' : 'angle'})
    elif variable == 'NDVI':
        da = da.rename({'band_data' : 'NDVI'})
    elif variable == 'SWI':
        da = da.rename({'band_data' : 'SWI'})
    else:
        da = da.rename({'band_data' : 'VV'})
    return da
                                      
data = xr.open_mfdataset(files, engine='rasterio', preprocess=preprocess_s1, join='override')

In [ ]:
monthly_mean = data.resample(time='1m').mean()
#yearly_mean = data.groupby('time.year').groupby('year.month').mean()

In [ ]:
df = yearly_mean.to_dataframe().reset_index()
df[['x','y']] = df[['x','y']].round(4)

In [ ]:
ymean = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.x,df.y))

In [ ]:
monthly_mean.NDVI.plot.contourf(x='x', y='y', col='month')

In [ ]:
monthly_mean.NDVI.plot(x='x', y='y', col='month', levels=[0.1,0.2,0.3,0.4,0.5,0.6,1])

In [ ]:
monthly_mean.NDVI.plot.contour(x='x', y='y', col='month', levels=[0.1,0.2,0.3,0.4,0.5,0.6,1])

In [ ]:
monthly_mean.NDVI.plot.contour(x='x', y='y', col='month', levels=7)

In [ ]:
monthly_mean.NDVI.plot.contour(x='x', y='y', col='month', levels=[0.1,0.2,3,0.5,0.6,1])

In [ ]:
import hvplot.xarray  # noqa

In [ ]:
monthly_mean


In [ ]:
monthly_mean['NDVI'].plot.contour(col='time', robust=True, col_wrap=12, cmap='CMRmap', levels=6)

In [ ]:
monthly_mean.NDVI.hvplot(col='month')

In [ ]:
ndvi = monthly_mean.hvplot(x='x', y='y', col='month',width=192, height=132, yaxis=False, xaxis=False, colorbar=False, cmap='viridis')
ndvi

## Extract pixels as a Numpy array

In [ ]:
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt

img = ee.Image('LANDSAT/LC08/C01/T1_SR/LC08_038029_20180810') \
  .select(['B4', 'B5', 'B6'])

aoi = ee.Geometry.Polygon(
  [[[-110.8, 44.7],
    [-110.8, 44.6],
    [-110.6, 44.6],
    [-110.6, 44.7]]], None, False)

rgb_img = geemap.ee_to_numpy(img, region=aoi)
print(rgb_img.shape)

In [ ]:
# Scale the data to [0, 255] to show as an RGB image. 
# Adapted from https://bit.ly/2XlmQY8. Credits to Justin Braaten
rgb_img_test = (255*((rgb_img[:, :, 0:3] - 100)/3500)).astype('uint8')
plt.imshow(rgb_img_test)
plt.show()

# Export Mean Data from Gee as array

In [2]:
import geemap
import ee
from irrigation_detection import add_ndvi
from irrigation_detection import mask_by_landcover

In [3]:
Map = geemap.Map(center=(49.939, 8.5), zoom=16)
Map.add_basemap('HYBRID')
Map

Map(center=[49.939, 8.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

In [17]:
# Draw any shapes on the map using the Drawing tools before executing this code block
feature = Map.draw_last_feature
roi = feature.geometry()
aoi = roi.buffer(10000)
Map.addLayer(aoi)

In [13]:
s1_collection = ee.ImageCollection("COPERNICUS/S1_GRD") \
    .filterBounds(roi) \
    .filterDate('2016-03-01', '2021-10-31') \
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV")) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .select(['VV', 'angle']) 

#Add ndvi band with 15 day offsett for s2 images
s1_collection = s1_collection.map(add_ndvi)

In [14]:
#Mask out area with invalid landcover class like build up areas
lc = ee.Image(ee.ImageCollection("ESA/WorldCover/v100").first().clip(aoi).select('Map'))
s1_collection = s1_collection.map(lambda x: x.addBands(lc))
s1_collection = s1_collection.map(mask_by_landcover)

In [16]:
Map.addLayer(s1_collection.first(), {'min': -30, 'max': 0, 'palette': ['red', 'blue']})